# AMPERIUM NNM (Trainer)
This notebook allows you to train a neural amp model based on a pair of input/output WAV files that you have of the amp you want to model. Amp + Cab and only Amp supported.

**To use this notebook**:
Go to [colab.research.google.com](https://colab.research.google.com/), select the "GitHub" tab, and select this notebook.

🔶**Before you run**🔶

Make sure to get a GPU! (Runtime->Change runtime type->Select "GPU" from the "Hardware accelerator dropdown menu)

⚠**Warning**⚠

Google Colab GPU instances only last for 12 hours.
Plan your training accordingly!

## Steps:
1. Install everything
2. Upload audio files
3. Settings
4. Run & Export
5. Download your files

## Step 1: Install
Install `AmperiumNNM` and the other Python packages it depends on.

In [ ]:
!git clone https://github.com/KirillDeLutherSergeev/AmperiumNNM.git '/content/AmperiumNNM'

%cd 'AmperiumNNM/'

from AudioUtils import *
from Training import *
from ExportNNM import *
import matplotlib.pyplot as plt
import os

## Step 2: Upload audio files
### Step 2A
Upload the input (DI) and output (reamped) files you want to use as input.wav and output.wav.

### Step 2B
Or copy files from your google drive to /content folder.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!cp '/content/drive/MyDrive/ML/TrainingSignalGuitarTX.wav' '/content/input.wav'
!cp '/content/drive/MyDrive/ML/5153B_ReactIR/ToneX_5153B_ReactIR.wav' '/content/output.wav'

## Step 3: Settings

In [ ]:
#@title Settings { display-mode: "form" }

#@markdown NNM file name
name = "EVH5153B_V1" #@param {type:"string"}
#@markdown Amp description
ampName = "EVH 5150III 50 Blue channel" #@param {type:"string"}
#@markdown Cab description
cabName = "None" #@param {type:"string"}
#@markdown Model type
modelType = "Amp + Cab" #@param ["Amp + Cab", "Amp"]

#@markdown Input file start offset in seconds, set to 0 to use offset in samples
offsetSeconds = 5 #@param {type:"number"}
#@markdown start offset in samples
offsetSamples = 0 #@param {type:"number"}
#@markdown Output file delay
delaySamples = 2 #@param {type:"number"}

#Training parameters
learningRate = 0.008
epsilon = 1e-08
#lossType = "mse" #@param ["mae", "mse"]
lossType = "mse"

#Dataset parameters
trainSize = 0.8

numSteps = 4096
overlap = 256
batchSize = 64

in_file = '/content/input.wav'
out_file = '/content/output.wav'

x_all, y_all = load_audio_data(in_file, out_file, offsetSeconds, offsetSamples, delaySamples)

x_train, y_train, x_test, y_test = prepare_dataset(x_all, y_all, numSteps=numSteps, trainSize=trainSize, overlap=overlap, batchSize=batchSize)

if modelType != "Amp":
  model = build_model(True, True, True, loss=lossType, learningRate=learningRate, epsilon=epsilon)
  modelTypeID = 11
else:
  model = build_model(False, True, False, loss=lossType, learningRate=learningRate, epsilon=epsilon)
  modelTypeID = 10

## Step 4: Run & Export

In [ ]:
#@title Step 4: Run & Export { display-mode: "form" }

#@markdown Epochs to train
epochs = 16 #@param {type:"integer"}

train_model(x_train, y_train, x_test, y_test, model, epochs)

export_model_to_nnm('/content/'+name+'.nnm', 1, 0, model, modelTypeID, ampName, cabName)

if os.path.exists('/content/drive/'):
    export_model_to_nnm('/content/drive/MyDrive/ML/5153B_ReactIR/'+name+'.nnm', 1, 0, model, modelTypeID, ampName, cabName)